In [65]:
import random
from typing import List, Tuple, Dict

# GA parameters
population_size = 100
mutation_rate = 0.01
crossover_rate = 0.8
max_generations = 500
alpha = None  # Penalty coefficient


 # Problem data (will be set when loading input)
n_items = 0
max_weight = 0
values = []
weights = []
value_weight_ratio = []


In [66]:
"""
        Optimized Genetic Algorithm for Knapsack Problem
        
        Args:
            population_size: Number of chromosomes in each generation
            mutation_rate: Probability of mutation for each gene
            crossover_rate: Probability of crossover between parents
            max_generations: Maximum number of generations to evolve
            alpha: Penalty coefficient for constraint violation
"""




def init_ga(pop_size=100, mut_rate=0.01, cross_rate=0.8, max_gen=500, penalty_alpha=None):
    global population_size, mutation_rate, crossover_rate, max_generations, alpha
    population_size = pop_size
    mutation_rate = mut_rate
    crossover_rate = cross_rate
    max_generations = max_gen
    alpha = penalty_alpha 


In [67]:
def load_input(filename: str):
    global n_items, max_weight, values, weights, value_weight_ratio, alpha
    try:
        with open(filename, 'r') as f:
                  # Read first line: number of items and max weight
            first_line = f.readline().strip().split()
            n_items = int(first_line[0])
            max_weight = int(first_line[1])

  # Read items (value, weight pairs)
            values = []
            weights = []

            for _ in range(n_items):
                line = f.readline().strip().split()
                values.append(int(line[0]))
                weights.append(int(line[1]))
    
                # Precompute value-to-weight ratios
            value_weight_ratio = [v / w for v, w in zip(values, weights)]

                # Set penalty coefficient if not provided
            if alpha is None:
                alpha = max(values) / max_weight * 10  # Strong penalty
                print(f"Setting penalty α = {alpha:.2f}")

            print(f"Loaded problem: {n_items} items, max weight: {max_weight}")
            print(f"Value range: {min(values)} - {max(values)}")
            print(f"Weight range: {min(weights)} - {max(weights)}")

    except FileNotFoundError:
        print(f"File {filename} not found!")
                # Create sample data for testing
        create_sample_data()

def create_sample_data():
    global n_items, max_weight, values, weights, value_weight_ratio, alpha
    print("Creating sample data for testing...")
    n_items = 4
    max_weight = 5
    values = [10, 40, 30, 50]
    weights = [5, 4, 6, 3]
    value_weight_ratio = [v / w for v, w in zip(values, weights)]
    if alpha is None:
        alpha = 20.0  # Strong penalty for sample data
    print(f"Sample problem: {n_items} items, max weight: {max_weight}")
    print(f"Items: {list(zip(values, weights))}")


In [68]:
def create_smart_chromosome() -> List[int]:
    """
 Create a chromosome using greedy heuristic as starting point
        """
        # Sort items by value-to-weight ratio descending
    sorted_indices = sorted(range(n_items), key=lambda i: value_weight_ratio[i], reverse=True)
    chromosome = [0] * n_items
    current_weight = 0


        # Greedily select items with best ratios that fit
    for i in sorted_indices:
        if current_weight + weights[i] <= max_weight:
            chromosome[i] = 1
            current_weight += weights[i]
  # Add some randomness - randomly flip some bits
    for i in range(n_items):
        if random.random() < 0.1:  # 10% chance to flip each bit
            chromosome[i] = 1 - chromosome[i]
    return chromosome

def create_chromosome() -> List[int]:
    if random.random() < 0.3:  # 30% smart initialization
        return create_smart_chromosome()
    else: # 70% random initialization but with bias toward lighter items --  Bias toward selecting lighter items (higher probability for lower weight)
        return [1 if random.random() < (0.3 + 0.4 * (1 - weights[i] / max(weights))) else 0
                for i in range(n_items)]

def calculate_fitness(chromosome: List[int]) -> float:
    """
     Calculate fitness with penalty for constraint violation
    Optimized version using list comprehensions and zip
    """
    total_value = sum(v * b for v, b in zip(values, chromosome))
    total_weight = sum(w * b for w, b in zip(weights, chromosome))
    if total_weight <= max_weight:
        return total_value
    else:
        return total_value - alpha * (total_weight - max_weight)


In [69]:
def repair_chromosome(chromosome: List[int]) -> List[int]:
    """
    Optimized repair of an infeasible chromosome
    """
    total_weight = sum(w * b for w, b in zip(weights, chromosome))
    if total_weight <= max_weight:
        return chromosome

    repaired = chromosome.copy()
    selected = [i for i in range(n_items) if repaired[i] == 1]
     # Sort selected items by value-to-weight ratio (worst first)
    selected.sort(key=lambda i: value_weight_ratio[i])
 # Remove items until feasible
    for i in selected:
        repaired[i] = 0
        total_weight -= weights[i]
        if total_weight <= max_weight:
            break
    return repaired

def crossover(parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
    """
    Optimized two-point crossover
    """
    if random.random() > crossover_rate:
        return parent1.copy(), parent2.copy()
    point1 = random.randint(1, n_items - 2)
    point2 = random.randint(point1 + 1, n_items - 1)
    c1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    c2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
    return c1, c2

def mutate(chromosome: List[int]) -> List[int]:
    """
    Optimized mutation implementation
    """
    mutated = chromosome.copy()
      # Bit-flip mutation
    for i in range(n_items):
        if random.random() < mutation_rate:
            mutated[i] = 1 - mutated[i]
             # Occasionally do swap mutation
    if random.random() < 0.1:
        i, j = random.sample(range(n_items), 2)
        mutated[i], mutated[j] = mutated[j], mutated[i]
    return mutated


In [70]:
def initialize_population() -> List[List[int]]:
    """Generate initial population with mix of strategies"""
    population = []
    for i in range(population_size):
        chrom = create_chromosome()
        if i < population_size // 4:
            chrom = repair_chromosome(chrom)
        population.append(chrom)
    return population

def selection(population, fitness_scores):
    """
        Tournament selection with optimized implementation
        """
    tournament_size = min(5, len(population))
    tournament = random.sample(list(zip(population, fitness_scores)), tournament_size)
    return max(tournament, key=lambda x: x[1])[0].copy()

def evolve_generation(population):
    """
    Optimized generation evolution
    """
       # Calculate fitness for all chromosomes
    fitness_scores = [calculate_fitness(ch) for ch in population]

          # Create new population
    new_pop = []

      # Elitism - keep best chromosomes
    elite_count = max(1, population_size // 10)  # Keep top 10%
    elites = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i], reverse=True)[:elite_count]
    new_pop.extend([population[i].copy() for i in elites])
     # Generate rest of population
    while len(new_pop) < population_size:
          # Select parents
        p1 = selection(population, fitness_scores)
        p2 = selection(population, fitness_scores)

           # Crossover
        c1, c2 = crossover(p1, p2)
        c1 = mutate(c1)
        c2 = mutate(c2)
           # Occasionally repair children
        if random.random() < 0.2:
            c1 = repair_chromosome(c1)
        if random.random() < 0.2:
            c2 = repair_chromosome(c2)
            # Add to new population
        new_pop.extend([c1, c2])
    return new_pop[:population_size]

def solve():
    """
    Run the Genetic Algorithm with optimized implementation
    """

    print(f"\nStarting Genetic Algorithm...")
    print(f"Population size: {population_size}")
    print(f"Mutation rate: {mutation_rate}")
    print(f"Crossover rate: {crossover_rate}")
    print(f"Max generations: {max_generations}")
    print(f"Penalty coefficient α: {alpha:.2f}")
        

    population = initialize_population()
    best_hist, avg_hist, feasible_hist = [], [], []
    best_fitness = float('-inf')
    best_solution = None
    no_improve = 0

    for gen in range(max_generations):
        fitness_scores = [calculate_fitness(ch) for ch in population]
        feasible_count = sum(1 for ch in population if sum(w*b for w,b in zip(weights,ch)) <= max_weight)
        best_gen = max(fitness_scores)
        avg_gen = sum(fitness_scores)/len(fitness_scores)

        best_hist.append(best_gen)
        avg_hist.append(avg_gen)
        feasible_hist.append(feasible_count)

        if best_gen > best_fitness:
            best_fitness = best_gen
            best_solution = population[fitness_scores.index(best_gen)].copy()
            no_improve = 0
        else:
            no_improve += 1

        if gen % 50 == 0 or gen == max_generations - 1:
            print(f"Gen {gen}: Best={best_gen:.1f}, Avg={avg_gen:.1f}, Feasible={feasible_count}/{population_size}")

        if no_improve > 100:
           
            print(f"Early stopping at generation {gen} (no improvement for {no_improve} generations)")
            break

        population = evolve_generation(population)

    if sum(w*b for w,b in zip(weights,best_solution)) > max_weight:
        best_solution = repair_chromosome(best_solution)

    best_value = sum(v*b for v,b in zip(values,best_solution))
    stats = {
        'generations_run': gen+1,
        'best_fitness_history': best_hist,
        'avg_fitness_history': avg_hist,
        'feasible_count_history': feasible_hist
    }
    return best_solution, best_value, stats


In [71]:
def print_solution(solution: List[int], total_value: int) -> None:
        """
        Print the solution with detailed analysis
        (Identical to original version)
        """
        print(f"\nBEST SOLUTION FOUND:")
        print(f"Total Value: {total_value}")
        print(f"Solution: {' '.join(map(str, solution))}")
        
        total_weight = sum(solution[i] * weights[i] for i in range(n_items))
        selected_items = [i for i in range(n_items) if solution[i] == 1]
        
        print(f"\nDetails:")
        print(f"Total Weight: {total_weight}/{max_weight}")
        print(f"Selected Items: {selected_items}")
        print(f"Weight utilization: {total_weight/max_weight*100:.1f}%")
        print(f"Constraint satisfied: {'Yes' if total_weight <= max_weight else 'No'}")
        
        if selected_items:
            print(f"\nSelected Items Details:")
            for i in selected_items:
                ratio = values[i] / weights[i]
                print(f"  Item {i}: value={values[i]}, weight={weights[i]}, ratio={ratio:.3f}")

def write_results_to_file(filename: str, solution: List[int], total_value: int, stats: Dict):
    """
    Append results to a file named filename + '.txt'.
    Creates file if it doesn't exist.
    """
    result_filename = f"{filename}Results.txt"
    try:
        with open(result_filename, 'a') as f:
            f.write("=" * 50 + "\n")
            f.write(f"Run Results for {filename}\n")
            f.write(f"Total Value: {total_value}\n")

            total_weight = sum(solution[i] * weights[i] for i in range(n_items))
            f.write(f"Total Weight: {total_weight}/{max_weight}\n")
            f.write(f"Solution: {' '.join(map(str, solution))}\n")

            selected_items = [i for i in range(n_items) if solution[i] == 1]
            f.write(f"Selected Items: {selected_items}\n")
            f.write(f"Weight utilization: {total_weight / max_weight * 100:.1f}%\n")
            f.write(f"Constraint satisfied: {'✅ Yes' if total_weight <= max_weight else '❌ No'}\n\n")

            if selected_items:
                f.write("Selected Items Details:\n")
                for i in selected_items:
                    ratio = values[i] / weights[i]
                    f.write(f"  Item {i}: value={values[i]}, weight={weights[i]}, ratio={ratio:.3f}\n")

            f.write(f"\nGenerations Run: {stats['generations_run']}\n")
            f.write(f"Best Fitness History (last 5): {stats['best_fitness_history'][-5:]}\n")
            f.write(f"Average Fitness History (last 5): {stats['avg_fitness_history'][-5:]}\n")
            f.write(f"Feasible Count History (last 5): {stats['feasible_count_history'][-5:]}\n")
            f.write("=" * 50 + "\n\n")

        print(f"✅ Results appended to {result_filename}")
    except Exception as e:
        print(f"❌ Error writing to {result_filename}: {e}")


In [72]:
def main():
    print("KNAPSACK PROBLEM - GENETIC ALGORITHM SOLVER")
    print("=" * 50)
    init_ga(200, 0.02, 0.8, 1000)
    filename = "ks_4_0"
    load_input(filename)
    best_solution, best_value, stats = solve()
    print_solution(best_solution, best_value)
    write_results_to_file(filename, best_solution, best_value, stats)


In [73]:
main()


KNAPSACK PROBLEM - GENETIC ALGORITHM SOLVER
Setting penalty α = 13.64
Loaded problem: 4 items, max weight: 11
Value range: 4 - 15
Weight range: 3 - 8

Starting Genetic Algorithm...
Population size: 200
Mutation rate: 0.02
Crossover rate: 0.8
Max generations: 1000
Penalty coefficient α: 13.64
Gen 0: Best=19.0, Avg=7.5, Feasible=162/200
Gen 50: Best=19.0, Avg=17.6, Feasible=194/200
Gen 100: Best=19.0, Avg=16.1, Feasible=188/200
Early stopping at generation 101 (no improvement for 101 generations)

BEST SOLUTION FOUND:
Total Value: 19
Solution: 0 0 1 1

Details:
Total Weight: 11/11
Selected Items: [2, 3]
Weight utilization: 100.0%
Constraint satisfied: Yes

Selected Items Details:
  Item 2: value=15, weight=8, ratio=1.875
  Item 3: value=4, weight=3, ratio=1.333
✅ Results appended to ks_4_0Results.txt
